<a href="https://colab.research.google.com/github/alicedhuynh/Rcode-for-STAT-560/blob/master/Improve_your_client's_Social_Media_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> Business Questions </center>

##Introduction

This file is a series of 16 questions that will allow you to answer business problems.


Load data from Google Drive

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
GlobalPage = pd.read_csv('/content/drive/My Drive/openclassrooms/data/GlobalPage.csv')
FansPerCountry = pd.read_csv('/content/drive/My Drive/openclassrooms/data/FansPerCountry.csv')
FansPerGenderAge = pd.read_csv('/content/drive/My Drive/openclassrooms/data/FansPerGenderAge.csv')
PostInsights = pd.read_csv('/content/drive/My Drive/openclassrooms/data/PostInsights.csv')
FansPerLanguage = pd.read_csv('/content/drive/My Drive/openclassrooms/data/FansPerLanguage.csv')
PopStats = pd.read_csv('/content/drive/My Drive/openclassrooms/data/PopStats.csv')
FansPerCity = pd.read_csv('/content/drive/My Drive/openclassrooms/data/FansPerCity.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Convert csv to database

In [3]:
import pandas as pd
import sqlite3
from sqlite3 import Error

# make a sql file
db_file = 'Huynh_Alice_1_database_122023.sql'
conn = sqlite3.connect(db_file)
print(f"Connected to database: {db_file}")

# Write the data to a sqlite tables

GlobalPage.to_sql('GlobalPage', conn, if_exists='replace', index=False)
FansPerCountry.to_sql('FansPerCountry', conn, if_exists='replace', index=False)
FansPerGenderAge.to_sql('FansPerGenderAge', conn, if_exists='replace', index=False)
PostInsights.to_sql('PostInsights', conn, if_exists='replace', index=False)
FansPerLanguage.to_sql('FansPerLanguage', conn, if_exists='replace', index=False)
PopStats.to_sql('PopStats', conn, if_exists='replace', index=False)
FansPerCity.to_sql('FansPerCity', conn, if_exists='replace', index=False)


Connected to database: Huynh_Alice_1_database_122023.sql


1260

In [4]:
# show first 5 rows for each dataframe
tables = ["GlobalPage","FansPerCountry","FansPerGenderAge",\
          "PostInsights","FansPerLanguage","PopStats","FansPerCity"]
for table in tables:
    df = pd.read_sql_query(f"SELECT * FROM {table} limit 5", conn)
    print(f"==={table}===")
    print(df)


===GlobalPage===
         Date CountryCode  NewLikes  DailyPostsReach  DailyPostShares  \
0  2018-09-19          AO      2458           512012           338773   
1  2018-09-19          BE      4319           899695           595285   
2  2018-09-19          BF     18130          3776513          2498736   
3  2018-09-19          BI      1330           277229           183429   
4  2018-09-19          BJ      5660          1179116           780164   

   DailyPageActions  DailyPostImpressions  
0             15910               1919541  
1             27958               3372971  
2            117355              14158199  
3              8614               1039337  
4             36641               4420523  
===FansPerCountry===
         Date CountryCode  NumberOfFans
0  2018-09-19          AO          5609
1  2018-09-19          BE          9856
2  2018-09-19          BF         41371
3  2018-09-19          BI          3037
4  2018-09-19          BJ         12917
===FansPerGenderAge

Write SQL query

In [5]:
def run_query(query):
    "function to write a query "
    df = pd.read_sql_query(f"{query}", conn)
    print(df)


##Section 1: Basic Summary Statistics

### Question 1: Unique Countries
How many unique countries are there?


In [8]:
unique_countries = '''
SELECT count(distinct CountryName)
FROM PopStats;
'''
run_query(unique_countries)

   count(distinct CountryName)
0                          150


### Question 2: Unique Cities
How many unique cities are there?

In [9]:
unique_cities = '''
SELECT count(distinct City)
FROM FansPerCity;
'''
run_query(unique_cities)

   count(distinct City)
0                    46


### Question 3: Unique Languages
How many unique languages are there?

In [10]:
unique_languages = '''
SELECT count(distinct Language)
FROM FansPerLanguage;
'''
run_query(unique_languages)

   count(distinct Language)
0                        41


### Question 4: Global Page Reach
What is the daily average reach of the posts (i.e. DailyPostsReach) on the global page over the period?


In [11]:
globalpagereach = '''
SELECT AVG(DailyPostsReach)
FROM GlobalPage;
'''
run_query(globalpagereach)

   AVG(DailyPostsReach)
0          1.862816e+06


### Question 5: Global Page Likes
What is the daily average engagement rate (i.e. NewLikes) on the global page over the period?  Round the result to 2 decimal places.


In [ ]:
globalpagelike = '''
SELECT round(AVG(NewLikes),2)
FROM GlobalPage;
'''
run_query(globalpagelike)


## Section 2: Location Analysis
In these queries, make sure you select the latest date in the main table.  E.g:

SELECT max(date) from FansPerCountry

Then use the result of this in the WHERE clause of your main query.




### Question 6: Top 10 countries by Number of Fans
What are the top 10 countries (considering the number of fans)?  Show a table of results containing the following columns:

*   CountryCode
*   CountryName   
*   NumberOfFans

In [12]:
Q6 = '''
SELECT t1.CountryCode, t2.CountryName, t1.NumberOfFans
FROM FansPerCountry t1
JOIN PopStats t2
ON t1.CountryCode = t2.CountryCode
WHERE DATE = (SELECT max(date) from FansPerCountry)
ORDER BY t1.NumberOfFans DESC
LIMIT 10
'''
run_query(Q6)


  CountryCode                       CountryName  NumberOfFans
0          CI                       Ivory Coast        112160
1          CM                          Cameroon        102211
2          SN                           Senegal         83561
3          FR                            France         73252
4          MG                        Madagascar         72956
5          CD  Democratic Republic of the Congo         50705
6          BF                      Burkino Faso         43500
7          ML                              Mali         40578
8          DZ                           Algeria         39093
9          GN                            Guinea         36821


### Question 7: Top 10 countries by Penetration Ratio
What are the top 10 countries by penetration ratio (i.e. the % of the country population that are fans)?  Show a table of results containing the following columns:

* CountryName
* PenetrationRatio
* NumberOfFans
* Population


In [13]:
Q7 = '''
SELECT
  t2.CountryName, ROUND(t1.NumberOfFans*100.0/t2.Population, 2) || ' %' as PenetrationRatio,
  t1.NumberOfFans, t2.Population
FROM FansPerCountry t1
JOIN PopStats t2 ON t1.CountryCode = t2.CountryCode
WHERE Date = (SELECT MAX(Date) FROM FansPerCountry)
ORDER BY PenetrationRatio DESC
LIMIT 10

'''
run_query(Q7)


        CountryName PenetrationRatio  NumberOfFans  Population
0           Reunion           2.41 %         20885      866506
1  French Polynesia           1.82 %          5148      283007
2     New Caledonia           1.79 %          5032      280460
3         Mauritius           1.77 %         24210     1364283
4        Martinique           1.44 %          5427      376480
5        Guadeloupe           1.36 %          5379      395700
6             Gabon           1.13 %         23954     2119036
7           Mayotte           0.73 %          1983      270372
8           Comoros            0.6 %          4925      821164
9     French Guiana           0.57 %          1687      296711


### Question 8: Bottom 10 Cities by Number of Fans
What are the bottom 10 cities (considering the number of fans) among countries with a population over 20 million?

This could be considered our growth potential.

Show a table of results containing the following columns:

* CountryName
* City
* NumberOfFans
* Population


In [14]:
Q8 = '''
SELECT t2.CountryName, t1.City, NumberOfFans, Population
FROM FansPerCity t1
JOIN PopStats t2
ON t1.CountryCode = t2.CountryCode
WHERE t1.Date = (SELECT MAX(Date) FROM FansPerCity) AND t2.Population > 20000000
ORDER BY NumberOfFans
LIMIT 10
'''
run_query(Q8)


   CountryName          City  NumberOfFans  Population
0      Algeria        Bejaia          2301    41657488
1     Cameroon    Ngaoundere          2318    25640965
2   Madagascar  Fianarantsoa          2366    25683610
3      Algeria    Tizi Ouzou          2524    41657488
4       Canada      Montreal          2887    35881659
5      Algeria          Oran          2920    41657488
6  Ivory Coast       Bouake           3376    24290000
7  Ivory Coast        Cocody          3647    24290000
8      Morocco    Casablanca          3951    34314130
9       Angola        Luanda          4614    30355880


## Section 3: Fan Analysis


### Question 9: Analysis by age group (split of fans)
What is the split of page fans across age groups (in %)?

Show a table of results containing the following columns:


In [15]:
Q9 = '''
SELECT AgeGroup, round(sum(NumberOfFans)*100/
	(
		SELECT SUM(NumberofFans)
		FROM FansPerGenderAge
		WHERE Date = (SELECT MAX(Date) FROM FansPerGenderAge)
	)
		, 2) || ' %' AS PercentageOfFans
FROM FansPerGenderAge
WHERE Date = (SELECT MAX(Date) FROM FansPerGenderAge)
GROUP BY AgeGroup
'''
run_query(Q9)

  AgeGroup PercentageOfFans
0    13-17            2.0 %
1    18-24           21.0 %
2    25-34           35.0 %
3    35-44           19.0 %
4    45-54            9.0 %
5    55-64            5.0 %
6      65+            6.0 %


### Question 10: Analysis by gender (split of fans)
What is the split of page fans by gender (in %)?

Show a table of results containing the following columns:

* Gender
* PercentageOfFans


In [16]:
Q10 ='''
SELECT Gender, ROUND ((SUM(NumberOfFans)*100.0/
  (
    SELECT SUM(NumberOfFans)
    FROM FansPerGenderAge
	WHERE Date = (SELECT MAX(Date) FROM FansPerGenderAge)
  )
), 2) || ' %' AS PercentageOfFans
FROM FansPerGenderAge
WHERE Date = (SELECT MAX(Date) FROM FansPerGenderAge)
GROUP BY Gender
'''
run_query(Q10)

  Gender PercentageOfFans
0      F          56.41 %
1      M           43.5 %
2      U           0.09 %


## Section 4: Language Analysis


### Question 11: English Speaking Fans
What is the number of the fans that have declared English as their primary language ?


In [17]:
Q11 = '''
SELECT SUM(NumberOfFans)
FROM FansPerLanguage
WHERE DATE = (SELECT max(Date) from FansPerCountry) and Language = 'en'
'''
run_query(Q11)

   SUM(NumberOfFans)
0              49418


### Question 12: English Speaking Fans Percentage

What is the percentage of the fans that have declared English as their primary language?


In [18]:
Q12 ='''
SELECT round((sum(NumberOfFans)*100.0/
  (
    SELECT sum(NumberOfFans)
    FROM FansPerLanguage
	WHERE Date = (SELECT MAX(Date) FROM FansPerLanguage)
  )
), 2) || ' %' AS Percentage
FROM FansPerLanguage
WHERE Language = 'en' and Date = (SELECT Max(Date) From FansPerLanguage)
'''
run_query(Q12)

  Percentage
0     5.09 %


### Question 13: Buying Power of English Speakers
Based on the number of fans who have declared English as their primary language and living in the US, what is the potential buying
power that can be accessed? (Please use the average income data per country for this question. It is estimated that on average,
0.01% of the annual income is dedicated to online magazine subscriptions in the US).


In [19]:
Q13 = '''
SELECT Language, '$ ' || ROUND(.0001*AverageIncome*numberoffans, 2) AS PotentialBuying
FROM FansPerLanguage t1
JOIN PopStats t2
ON t1.CountryCode = t2.CountryCode
WHERE CountryName = 'United states' AND t1.Date =(SELECT Max(Date) From FansPerLanguage)
GROUP BY Language
'''
run_query(Q13)

  Language PotentialBuying
0       en     $ 200322.75


## Section 5: Fan Engagment


### Question 14: Engagement per day of the week
What is the split of the EngagedFans across the days of the week (monday, tuesday,...)?

Give the result as a table with the following columns:

* DayOfWeek
* PercentageSplit

Based on the results, what is the best day of the week to publish posts?

Saturday would be the best week to publish.


In [ ]:
Q14 = '''
SELECT
  CASE strftime('%w', CreatedTime)
    WHEN '0' THEN 'Sunday'
    WHEN '1' THEN 'Monday'
    WHEN '2' THEN 'Tuesday'
    WHEN '3' THEN 'Wednesday'
    WHEN '4' THEN 'Thursday'
    WHEN '5' THEN 'Friday'
    WHEN '6' THEN 'Saturday'
  END AS  DayOfWeek,
  ROUND
  (
    SUM(EngagedFans)*100.0/
    (
    SELECT SUM(EngagedFans)
    FROM PostInsights
    ),2
  ) || ' %' AS PercentageSplit
FROM PostInsights
GROUP BY DayOfWeek
ORDER BY
  CASE DayOfWeek
   WHEN 'Sunday' THEN 0
   WHEN 'Monday' THEN 1
   WHEN 'Tuesday' THEN 2
   WHEN 'Wednesday' THEN 3
   WHEN 'Thursday' THEN 4
   WHEN 'Friday' THEN 5
   WHEN 'Saturday' THEN 6
  END
'''
run_query(Q14)


   DayOfWeek PercentageSplit
0     Sunday         12.08 %
1     Monday         19.23 %
2    Tuesday         18.67 %
3  Wednesday         15.38 %
4   Thursday          6.32 %
5     Friday          8.59 %
6   Saturday         19.73 %


In [ ]:
Q15 ='''
SELECT
  CASE
    WHEN strftime('%H:%M', CreatedTime) BETWEEN '05:00' AND '08:59' THEN '05:00 - 08:59'
    WHEN strftime('%H:%M', CreatedTime) BETWEEN '09:00' AND '11:59' THEN '09:00 - 11:59'
    WHEN strftime('%H:%M', CreatedTime) BETWEEN '12:00' AND '14:59' THEN '12:00 - 14:59'
    WHEN strftime('%H:%M', CreatedTime) BETWEEN '15:00' AND '18:59' THEN '15:00 - 18:59'
    WHEN strftime('%H:%M', CreatedTime) BETWEEN '19:00' AND '21:59' THEN '19:00 - 21:59'
   ELSE '22:00 or later'
    END AS TimeRange,
    ROUND
    (
      SUM(EngagedFans)*100.0/
      (
        SELECT SUM(EngagedFans)
        FROM PostInsights
      ), 2
     ) || ' %' AS Percentage

FROM PostInsights
GROUP BY TimeRange
'''
run_query(Q15)

        TimeRange Percentage
0   05:00 - 08:59    33.96 %
1   09:00 - 11:59    15.34 %
2   12:00 - 14:59     12.7 %
3   15:00 - 18:59     17.4 %
4   19:00 - 21:59    13.12 %
5  22:00 or later     7.48 %


### Question 15: Engagement per time of day
What is the split of the EngagedFans by time of the day?  Split the day into the following time ranges (call the column TimeRange):

* 05:00 - 08:59
* 09:00 - 11:59
* 12:00 - 14:59
* 15:00 - 18:59
* 19:00 - 21:59
* 22:00 or later

Give the result as a table with the following columns:

* TimeRange
* Percentage

Based on the results, what is the best time of the day to publish posts?

The best time to publish posts is morning time from 5:00 to 8:59.


In [ ]:
  Q15 ='''
SELECT
  CASE
    WHEN strftime('%H:%M', CreatedTime) BETWEEN '05:00' AND '08:59' THEN '05:00 - 08:59'
    WHEN strftime('%H:%M', CreatedTime) BETWEEN '09:00' AND '11:59' THEN '09:00 - 11:59'
    WHEN strftime('%H:%M', CreatedTime) BETWEEN '12:00' AND '14:59' THEN '12:00 - 14:59'
    WHEN strftime('%H:%M', CreatedTime) BETWEEN '15:00' AND '18:59' THEN '15:00 - 18:59'
    WHEN strftime('%H:%M', CreatedTime) BETWEEN '19:00' AND '21:59' THEN '19:00 - 21:59'
    WHEN strftime('%H:%M', CreatedTime) BETWEEN '22:00' AND '23:59' THEN '22:00 - 23:59'
    END AS TimeRange,
    ROUND
    (
      SUM(EngagedFans)*100.0/
      (
        SELECT SUM(EngagedFans)
        FROM PostInsights
      ), 2
     ) || ' %' AS Percentage

FROM PostInsights
GROUP BY TimeRange
'''
run_query(Q15)


       TimeRange Percentage
0  05:00 - 08:59    33.96 %
1  09:00 - 11:59    15.34 %
2  12:00 - 14:59     12.7 %
3  15:00 - 18:59     17.4 %
4  19:00 - 21:59    13.12 %
5  22:00 - 23:59     7.48 %


## Section 6: Optional Challeging Queries

### Question 16: Month to month change in engagement
Compute the change in PostClicks, EngagedFans and Reach from one month to the next.
Give the result as a table with the following columns:

* FromMonth
* ToMonth
* DeltaPostClicks
* DeltaEngagedFans
* DeltaReach

Where the deltas are the percentage differences between a month and its previous month.


In [ ]:
Q16 = '''
SELECT  strftime('%m', CreatedTime) as FromMonth,
  Lead(strftime('%m', CreatedTime)) OVER (Order by strftime('%m', CreatedTime)) AS ToMonth,
  ROUND(SUM(PostClicks)*1.0*100.0 / LEAD(SUM(PostClicks)) OVER (ORDER BY strftime('%m', CreatedTime)) - 1, 2) || '%' AS DeltaPostClicks,
  ROUND(SUM(EngagedFans)*1.0*100.0 / LEAD(SUM(EngagedFans)) OVER (ORDER BY strftime('%m', CreatedTime)) - 1, 2) || '%' AS DeltaEngagedFans,
  ROUND(SUM(Reach)*100.0 / LEAD(SUM(Reach)) OVER (ORDER BY strftime('%m', CreatedTime)) - 1, 2) || '%' AS DeltaReach
FROM PostInsights
GROUP BY strftime('%m', CreatedTime)
'''
run_query(Q16)

   FromMonth ToMonth DeltaPostClicks DeltaEngagedFans DeltaReach
0         01      02          16.74%           24.67%     23.18%
1         02      03          11.52%           14.33%     12.71%
2         03      04         165.68%          122.72%    145.36%
3         04      05          73.21%           87.39%     81.48%
4         05      06         113.65%           94.66%      98.7%
5         06      07          91.15%          100.26%     97.87%
6         07      08          76.63%            84.4%     81.02%
7         08      09           71.3%           66.63%     66.53%
8         09      10          21.79%           25.25%     21.18%
9         10      11         711.03%          638.27%    774.94%
10        11      12          96.94%           94.76%     88.62%
11        12    None            None             None       None
